In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [18]:
#week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Jul_list = [2, 9, 16, 23, 30]

In [19]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [20]:
len(link_day_minute_Jul_list)

14400

In [21]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [22]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [23]:
x[:,:2]

matrix([[ 1282.49031214,  1234.29593776],
        [ 1192.02182961,  1213.71923214],
        [ 1472.5629721 ,  1436.70679378],
        [ 1495.04134971,  1483.06576028],
        [ 1622.07219442,  1603.52930864],
        [ 1460.0047391 ,  1390.76279883],
        [ 1260.0655806 ,  1254.98473764],
        [ 1272.76960344,  1315.77393085],
        [ 1359.355558  ,  1388.03683058],
        [ 1657.71363566,  1603.15312666],
        [ 1532.3011974 ,  1544.73602489],
        [ 1150.57500212,  1198.16675689],
        [ 1196.3883539 ,  1148.67307418],
        [ 1361.80820092,  1298.95813595],
        [ 1287.23112097,  1229.97415679],
        [ 1020.1848718 ,   441.26046548],
        [ 1203.98924648,  1254.45432884],
        [ 1272.52641937,  1296.97988453],
        [ 1505.25618448,  1462.39140628],
        [  801.87718366,   922.25979863],
        [ 1307.55706654,  1165.03410251],
        [ 1410.96350677,  1333.28527304],
        [ 1271.02248225,  1212.2609458 ],
        [ 1302.59286008,  1264.851

In [24]:
np.size(A,0), np.size(A,1)

(24, 140)

In [25]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e+01, 2e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 27
 AA' NZ     : 3.510e+02
 Factor NZ  : 3.780e+02
 Factor Ops : 6.930e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.71402661e+07  0.00000000e+00  7.06e+03 1.21e+03  1.00e+06     0s
   1  -8.28724929e+06 -1.18052812e+07  3.46e+02 5.94e+01  1.62e+05     0s
   2  -7.20362423e+06 -8.36478964e+06  2.45e+01 4.20e+00  2.62e+04     0s
   3  -7.24852663e+06 -7.41975059e+06  2.45e-05 4.20e-06  3.0

In [26]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [27]:
lam_list

[5.556154903070189e-12,
 2646.052433425533,
 6.098827226346974e-12,
 0.0011503002416746108,
 0.17808716060451957,
 0.0004680521742425267,
 1.840429697016467e-09,
 655.1904793807379,
 0.007049327313584985,
 1291.6406603108294,
 152.02955006614798,
 0.007041730793029832,
 0.0004389075858942,
 0.0004416406059270881,
 591.863146554959,
 0.43574310526224325,
 1023.1113392984582,
 150.5551144107009,
 959.8765175436272,
 0.00046815380927647116,
 0.0004677290647664041,
 0.00011648832217857601,
 1309.0684658098749,
 262.58161491480547,
 0.0010291789789719438,
 807.6483675209329,
 0.0010395900935901703,
 1167.4818666377948,
 393.6700957140976,
 763.0782055670676,
 239.22885378389773,
 0.020505539182902766,
 6.700537982149476e-05,
 1117.9639106620975,
 325.5736631594198,
 992.3313806410487,
 1.8373720751088563,
 258.1555350431786,
 2.8635049273914746,
 1404.0779921146861,
 791.0952943913877,
 256.8808517542495,
 1.0949684176273151e-05,
 1.0949673227608108e-05,
 1.094966135069714e-05,
 225.9868050